<a href="https://colab.research.google.com/github/jaya-shankar/education-impact/blob/master/load_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf education-impact
!rm education-impact

rm: cannot remove 'education-impact': No such file or directory


In [ ]:
!git clone https://github.com/jaya-shankar/education-impact.git


In [4]:
root = "education-impact/" 
datasets_path = {
                    "infant_mortality"              :  root+ "datasets/Infant_Mortality_Rate.csv",
                    "child_mortality"               :  root+ "datasets/child_mortality_0_5_year_olds_dying_per_1000_born.csv",
                    "children_per_woman"            :  root+ "datasets/children_per_woman_total_fertility.csv",
                    "co2_emissions"                 :  root+"datasets/co2_emissions_tonnes_per_person.csv",
                    "population"                    :  root+ "datasets/converted_pop.csv",
                    "food_supply"                   :  root+ "datasets/food_supply_kilocalories_per_person_and_day.csv",
                    "gdp_per_captia"                :  root+ "datasets/gdp_per_capita_yearly_growth.csv",
                    "gini_index"                    :  root+ "datasets/gini.csv",
                    "life_expectancy"               :  root+ "datasets/life_expectancy_years.csv",
                    "malnutrition"                  :  root+ "datasets/malnutrition_weight_for_age_percent_of_children_under_5.csv",
                    "poverty_index"                 :  root+ "datasets/mincpcap_cppp.csv",
                    "maternal_mortality"            :  root+ "datasets/mmr_who.csv",
                    "people_in_poverty"             :  root+ "datasets/number_of_people_in_poverty.csv",
                    "primary_completion"            :  root+ "datasets/primary_school_completion_percent_of_girls.csv",
                    "ratio_b/g_in_primary"          :  root+ "datasets/ratio_of_girls_to_boys_in_primary_and_secondary_education_perc.csv",
                    "wcde-15--24"                   :  root+ "datasets/wcde-15--24.csv",
                    "wcde-25--34"                   :  root+ "datasets/wcde-25--34.csv",
                    "wcde-35--44"                   :  root+ "datasets/wcde-35--44.csv",
                    "wcde-45--54"                   :  root+ "datasets/wcde-45--54.csv",
                    "wcde-55--64"                   :  root+ "datasets/wcde-55--64.csv",
                    "wcde-65--74"                   :  root+ "datasets/wcde-65--74.csv",
                    "wcde-75--84"                   :  root+ "datasets/wcde-75--84.csv",
                    "wcde-85--94"                   :  root+ "datasets/wcde-85--94.csv",
                    "wcde-95--"                     :  root+ "datasets/wcde-95--.csv",
                 
                }

In [ ]:
datasets_path

In [45]:
import pandas as pd
import numpy as np

In [15]:
countries_arr = []
for path in datasets_path:
  df = read_csv(datasets_path[path])
  print(f"{'Factor: ' + path:<30} count: {len(set(df.Country.unique()))}")
  

Factor: infant_mortality       count: 266
Factor: child_mortality        count: 197
Factor: children_per_woman     count: 202
Factor: co2_emissions          count: 194
Factor: population             count: 197
Factor: food_supply            count: 179
Factor: gdp_per_captia         count: 221
Factor: gini_index             count: 195
Factor: life_expectancy        count: 195
Factor: malnutrition           count: 156
Factor: poverty_index          count: 195
Factor: maternal_mortality     count: 184
Factor: people_in_poverty      count: 145
Factor: primary_completion     count: 195
Factor: ratio_b/g_in_primary   count: 200
Factor: wcde-15--24            count: 202
Factor: wcde-25--34            count: 202
Factor: wcde-35--44            count: 202
Factor: wcde-45--54            count: 202
Factor: wcde-55--64            count: 202
Factor: wcde-65--74            count: 202
Factor: wcde-75--84            count: 202
Factor: wcde-85--94            count: 202
Factor: wcde-95--              cou

from the above output
- **malnutrition & people in povery** have least no of countries
- **infant mortality & gdp per captia** have highest no of countries

*Doubt:* Does having more data for one factor will make the decision tree bias?


###Steps
1. create a csv file such that each row contains all values of particular year & country present
2. the output for each row is year + 40 years corresponding value 
    1. **outputs** - life expectany, education level, gdp




In [53]:
PREDICT_FUTURE = 40
OUTPUTS = ['life_expectancy', 'gdp_per_captia', 'primary_completion' ]

In [ ]:

countries = list(read_csv('education-impact/datasets/Infant_Mortality_Rate.csv').Country.unique())
years = [y for y in range(1960,2015-PREDICT_FUTURE+1)]

In [ ]:
keys=[]
for y in years:
  for c in countries:
    keys.append((c,str(y)))

In [60]:
big_dic = {k : [] for k in keys}
for path in datasets_path:
  df = pd.read_csv(datasets_path[path])
  df.set_index("Country", inplace=True)
  for k in keys:
    try:
      big_dic[k].append(df.loc[k[0]][k[1]])
    except:
      big_dic[k].append(np.NaN)
 

In [49]:
print(len(datasets_path))

24


In [61]:
for output_path in OUTPUTS:
  df = pd.read_csv(datasets_path[output_path])
  df.set_index("Country", inplace=True)
  for k in keys:
    try:
      big_dic[k].append(df.loc[k[0]][str(int(k[1])+PREDICT_FUTURE)])
    except:
      big_dic[k].append(np.NaN)


In [62]:
columns = [k for k in datasets_path ]
output_columns = ["o_"+o for o in OUTPUTS]
columns.extend(output_columns)

In [ ]:
input_df = pd.DataFrame.from_dict(big_dic,orient='index', columns = columns)
modified_input = input_df.dropna(subset=["o_"+o for o in OUTPUTS])
modified_input2 = input_df.dropna(subset=["o_"+o for o in OUTPUTS],how='all')
print(input_df.info())
print(modified_input.info())
print(modified_input2.info())

From above output
- if we dont drop any rows our table size = 4256 entries
- if we drop rows containing any if all of outputs missing then our table size = 3039 entries
- if we drop rows containing any one of output missing then our table size = 1745 entries

so, I think its is better to go with second choice and build different models, but not sure it will not effect performance of the model


now we have the dataframe containing both inputs and ouputs,our next step is
1. split the data into train & test data
2. build DF model using tensorflow
3. check the accuracy of the model